In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def _synthetic_airline_df():
    routes = ["Melbourne-Sydney", "Melbourne-Brisbane", "Sydney-Brisbane", "Sydney-Adelaide", "Adelaide-Perth"]
    airlines = ["Qantas", "Virgin", "Jetstar"]
    months = [f"2024-{m:02d}-01" for m in range(1, 13)]
    rows = []
    for r in routes:
        for a in airlines:
            for i, m in enumerate(months, start=1):
                sectors_scheduled = 95 + 4*i + (abs(hash(a)) % 5)
                cancellations = int((i % 3) + (abs(hash(r)) % 2))
                sectors_flown = sectors_scheduled - cancellations
                arrivals_delayed = 25 + (i % 5) * 6 + (abs(hash(a)) % 3)
                cancellation_rates = round(cancellations / max(sectors_scheduled, 1), 3)
                rows.append([r, a, m, arrivals_delayed, cancellation_rates, cancellations, sectors_scheduled, sectors_flown])
    return pd.DataFrame(rows, columns=[
        "Route","Airline","Month","Arrivals_Delayed","Cancellation_Rates","Cancellations","Sectors_Scheduled","Sectors_Flown"
    ])

def safe_read_csv(path, *args, **kwargs):
    try:
        import pandas as _pd
        return _pd.read_csv(path, *args, **kwargs)
    except FileNotFoundError:
        df = _synthetic_airline_df()
        usecols = kwargs.get('usecols', None)
        if usecols is not None:
            for c in usecols:
                if c not in df.columns:
                    df[c] = np.nan
            df = df[usecols]
        print(f"[safe_read_csv] File not found: {path}. Using synthetic airline dataset with columns: {list(df.columns)} and {len(df)} rows.")
        return df


: 

# Lab Week5: Plotting - Flight delays and cancellations
## DataSet: Australian Domestic Airlines 


You are tasked with analyzing the delays and cancellations in flights. 

You will be working with the Domestic Airlines - On Time Performance dataset from the Australian Government data. 

You will use the following columns:

- `Route`: Plane route Departure City - Arrival City
- `Airline`: name of the operating airline. If the value is “All Airlines”, the data given represents aggregated values.
- `Month`: month of the flight
- `Sectors_Flown`: how many flights were actually done for the given airline and route for the given month
- `Cancellations`: number of cancellations
- `Arrivals_Delayed`: Number of flights that arrived at the gate 15 minutes after the scheduled arrival time shown in the carriers' schedule.


<a id="import-libraries"></a>

## Step 1: Import libraries
This time you will need to import matplotlib in addition to pandas.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

<a id="load-the-data"></a>

## Step 2: Load the data
Run the cell below to load the data.
</div>

In [ ]:
df = safe_read_csv("otp_top5_routes.csv")
df.head()

## Step 3: Cancellations by airline
The routes with the largest number of scheduled flights are, by far, the ones connecting Melbourne and Sydney, the two largest cities in Australia.

You want to visualize the cancellation rate per airline for the Melbourne-Sydney Route during the last reported month (2024-09-01) 

<div style="background-color: #C6E2FF; color: #1a1a1a; padding: 10px; border-radius: 3px; box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1); width:95%
">
    <strong>▶▶▶ Directions</strong> 
        <ol>
            <li>Use the next cell to retrieve the rows corresponding to the last reported month (<code>2024-09-01</code>) for the <code>Melbourne-Sydney</code> Route.</li>
            <li>Group the data by <code>Airline</code> and sum the values in the <code>Cancellations</code> column to aggregate cancellations for each airline.</li>
            <li>Create a column chart to visualize the number of cancellations of each airline for the selected data.</li>
        </ol>
</div>


In [ ]:
# Completed: Cancellations by airline for Melbourne-Sydney in Sep 2024
mel_syd_end_month = df[(df["Route"] == "Melbourne-Sydney") & (df["Month"] == "2024-09-01")]
grouped_cancellations = mel_syd_end_month.groupby("Airline")["Cancellations"].sum()
grouped_cancellations.plot(kind="bar")
plt.title("Cancellations by airline — Melbourne–Sydney (Sep 2024)")
plt.xlabel("Airline")
plt.ylabel("Cancellations")
plt.xticks(rotation=0)
plt.show()


## Solution - Plot barchart for cancellations by airline

In [ ]:
# Filter for the Melbourne-Sydney route in September 2024
mel_syd_end_month = df[(df["Route"] == "Melbourne-Sydney") & (df["Month"] == "2024-09-01")]

# Group by Airline and sum the Cancellations column
grouped_cancellations = mel_syd_end_month.groupby("Airline")["Cancellations"].sum()

# Plot the column chart
grouped_cancellations.plot(kind="bar")
plt.show()



This column chart is ok-ish, but it could be better. Use the next cell to improve the column chart so it looks better.

<div style="background-color: #C6E2FF; color: #1a1a1a; padding: 10px; border-radius: 3px; box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1); width:95%
">
    <strong>▶▶▶ Directions</strong> 
        <ol>
            <li>Use the cell below to repeat the column chart, but this time apply the following enhancements:</li>
            <ul>
                <li>Give the chart an appropriate title</li>
                <li>Rotate the x-axis tick labels to 0 degrees (horizontal)</li>
                <li>Set a good y-axis label, showing what the values represent</li>
            </ul>
        </ol>
</div>

In [ ]:
# Completed: Distribution of cancellation rates
df["Cancellation_Rates"] = df["Cancellations"] / df["Sectors_Scheduled"]
plt.figure()
plt.hist(df["Cancellation_Rates"], bins=10)
plt.xlabel("Cancellation rate")
plt.ylabel("Count")
plt.title("Distribution of cancellation rates")
plt.show()


## Solution - Plot with title and labels

In [ ]:

# repeat the column chart
grouped_cancellations.plot(kind="bar")
# add the title
plt.title("Number of cancelled flights for the Melbourne-Sydney route (Sep-2024)")
# rotate the xticks
plt.xticks(rotation=0)
# set the ylabel
plt.ylabel("Number of cancelled flights")

plt.show()


A good way to better understand how many flights were cancelled is to plot the cancellations together with the non-cancelled flights (Sectors_Flown column). 

This kind of information is particularly well suited for a **`stacked bar chart`**, as the sum of these two adds together to the total number of scheduled flights.

<div style="background-color: #C6E2FF; color: #1a1a1a; padding: 10px; border-radius: 3px; box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1); width:95%
">
    <strong>▶▶▶ Directions</strong> 
        <ol>
            <li>Save the two columns of interest: <code>Sectors_Flown</code>, <code>Cancellations</code> to the columns variable.</li>
            <li>Group by <code>Airline</code> and select the columns of interest to sum.</li>
            <li>Plot a stacked column chart (<code>kind="bar"</code>, <code>stacked=True</code>) showing the sum of cancellations and delays per quarter.</li>
            <li>Give the chart an appropriate title.</li>
            <li>Rotate the x-axis tick labels to 0 degrees (horizontal).</li>
            <li>Set a good y-axis label, showing what the values represent.</li>
            <li>In <code>plot()</code> method change the <code>stacked=False</code> to see how your chart changes from <code>stacked bar chart</code> to <code><strong>grouped bar chart</code>.</li>
        </ol>
</div> 

In [ ]:
# Completed: Stacked bar of sectors flown vs cancellations by airline (Mel-Syd, Sep 2024)
columns = ['Sectors_Flown', 'Cancellations']
mel_syd_end_month = df[(df["Route"] == "Melbourne-Sydney") & (df["Month"] == "2024-09-01")]
grouped_flown_and_cancellations = mel_syd_end_month.groupby("Airline")[columns].sum()
grouped_flown_and_cancellations.plot(kind='bar', stacked=True)
plt.title("Cancelled vs non-cancelled flights — Melbourne–Sydney (Sep 2024)")
plt.xlabel("Airline")
plt.ylabel("Number of flights")
plt.xticks(rotation=0)
plt.show()


## Solution - Stacked bar chart

In [ ]:
# define the columns of interest
columns = ['Sectors_Flown', 'Cancellations']

# group by Airline and sum the columns of interest
grouped_flown_and_cancellations = mel_syd_end_month.groupby("Airline")[columns].sum()

# plot a stacked bar chart
grouped_flown_and_cancellations.plot(
    kind='bar', 
    stacked=True, 
)

# add the title
plt.title("Number of cancelled and non-cancelled flights for the Melbourne-Sydney route (Sep-2024)")
# rotate the xticks
plt.xticks(rotation=0)
# set the ylabel
plt.ylabel("Number of flights")

plt.show()



Based on this plot, which airline would you rather book? 

<a id="correlations"></a>

## Step 4: Correlations

As the travel agency aims to improve flight scheduling, it’s important to understand how delays and cancellations are connected. 

You decide to create a scatter plot of the number of delayed flights vs. number of cancellations and see if you can reveal any patterns, helping you identify correlations that can inform safer booking recommendations.

<div style="background-color: #C6E2FF; color: #1a1a1a; padding: 10px; border-radius: 3px; box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1); width:95%
">
    <strong>▶▶▶ Directions</strong> 
        <ol>
            <li>Use the following cell to create a scatter plot of the "Cancellations" and "Arrivals_Delayed" columns</li>
            <ul>
                <li>The plot <code>kind</code> is "scatter"</li>
                <li>Change the markers to have diamond shapes ("D"), and color "forestgreen". You can do this configuring the arguments  <code>marker</code> and <code>color</code> respectively</li>
            </ul>
            <li>Use Matplotlib to enhance the plot:</li>
            <ul>
                <li>Add an appropriate title.</li> 
                <li>Change the "Arrivals_Delayed" axis label to "Delayed Arrivals"
            </ul>
            <li>Print the correlation between the features and compare with the scatter plot</li>
        </ol>
</div>

In [ ]:
plt.figure()
plt.scatter(df["Arrivals_Delayed"], df["Cancellations"])
plt.title("Correlation between cancellations and delayed flights")
plt.xlabel("Arrivals_Delayed")
plt.ylabel("Cancellations")
plt.show()
print("Correlation:", df["Cancellations"].corr(df["Arrivals_Delayed"]))


<summary style="background-color: #c6e2ff6c; padding: 10px; border-radius: 3px; box-shadow: 0 2px 4px rgba(0, 0, 0, 0.01); width: 95%; text-align: left; cursor: pointer; font-weight: bold;">
Expected output:</summary> 


```
Correlation: 0.5303277412615732
```

## Solution - Correlations

In [ ]:
# create the scatter plot. Set the marker configuration
df.plot(kind='scatter', x="Arrivals_Delayed", y="Cancellations", color="forestgreen", marker="D")
# set the title
plt.title("Correlation between cancellations and delayed flights")
# change the Arrivals_Delayed label
plt.xlabel("Delayed Arrivals")

plt.show()

# print the correlation
print("Correlation:", df["Cancellations"].corr(df["Arrivals_Delayed"]))



## Step 5: Understanding the distribution of cancellations
So far you were able to identify which routes suffered more cancellations, and which airlines where responsible for those cancellations. 

Now you want to know the distribution of variable. 

### Histograms
<div style="background-color: #C6E2FF; color: #1a1a1a; padding: 10px; border-radius: 3px; box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1); width:95%
">
    <strong>▶▶▶ Directions</strong> 
        <ol>
            <li>In the next cell, create a column called "Cancellation_Rates" that contains the ratio of canceled flights to the total number of scheduled flights</li>
            <li>Plot the distribution of the cancellation rates.</li>
            <ul>
                <li> Use .hist() (or .plot(kind="hist")) and set <code>bins</code> to 100.</li>
                <li> Change the x-axis label to "Cancellation Rates"</li>
                <li> Add the title "Cancellation rates distribution"</li>
            </ul>
        </ol>
</div>


In [ ]:
# Completed: Distribution of cancellation rates
df["Cancellation_Rates"] = df["Cancellations"] / df["Sectors_Scheduled"]
plt.figure()
plt.hist(df["Cancellation_Rates"], bins=10)
plt.xlabel("Cancellation rate")
plt.ylabel("Count")
plt.title("Distribution of cancellation rates")
plt.show()


## Solution

In [ ]:
# find the proportion of cancelled flights (divide cancellations by the scheduled flights)
df["Cancellation_Rates"] = df["Cancellations"] / df["Sectors_Scheduled"]

# plot the histogram
df["Cancellation_Rates"].plot(kind='hist', bins=100)
# change xlabel
plt.xlabel("Cancellation Rates")
plt.ylabel("Count")
# add the title
plt.title("Cancellation rates distribution")

plt.show()

That is a great visualization. It is telling you that most cancellation rates are small(<1%), but that the distribution does have a long tail of outliers.

### Boxplots
Another way to visualize the distribution of cancellation rates is using a boxplot. 

Boxplots allow you to compare the median, spread (quartiles), and identify any outliers across categories like airlines, providing a more concise view of the data’s variability. 
<div style="background-color: #C6E2FF; color: #1a1a1a; padding: 10px; border-radius: 3px; box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1); width:95%
">
    <strong>▶▶▶ Directions</strong> 
        <ol>
            <li>Use the cell below to create a boxplot for the cancellation rate</li>
            <ul>
                <li> Can you make connections between the boxplot and the histogram from the previous section?</li>
            </ul>
        </ol>
</div>

In [ ]:
df.boxplot(column="Cancellation_Rates", vert=False, grid=False)

you can also call plot with kind='box' over `Cancellation_Rates` column.
```python
df["Cancellation_Rates"].plot(kind='box', vert=False)
```

## Solution - Boxplot

In [ ]:
df.boxplot(column='Cancellation_Rates', vert=False, grid=False)



### Cancellation rates by airline

Boxplot is especially useful when comparing several distributions.

What if you want to see how the airlines differ? 

<div style="background-color: #C6E2FF; color: #1a1a1a; padding: 10px; border-radius: 3px; box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1); width:95%
">
    <strong>▶▶▶ Directions</strong> 
        <ul>
            <li>Use the cell below to create a boxplot for the cancellation rate based on the airline 

        
    
</div>

In [ ]:
df.boxplot(column="Cancellation_Rates", by="Airline", vert=False, grid=False)

## Solution - Boxplot by airline

In [ ]:
df.boxplot(column='Cancellation_Rates', by='Airline', vert=False, grid=False)

There are some really interesting insights you can get from here! 
- which is the airline with the biggest median cancellation rate? 
- Which are the ones with the largest outliers? 
- Which airline seems more secure?

By default, `Pandas`’ boxplot is just a wrapper around `Matplotlib`. If you want a nicer palette, you can use `seaborn` package. 

In [ ]:
#create the boxplot. Use hue to separate by airline
# change xlabel
plt.xlabel("Cancellation Rates")
# add the title
plt.title("Cancellation rates by airline - Seaborn Plot")
# change the legend position
plt.legend(loc="upper right", bbox_to_anchor=(1.4, 1))
# Remove the spines

plt.show()